# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head(3)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11


In [4]:
q_prod_cus = data.groupby(['CustomerID','ProductName']).sum()
q_prod_cus.head(3)

SalesID  ProductID  Quantity
CustomerID ProductName                                          
33         Apricots - Dried          172592        324         1
           Assorted Desserts        5363705        219         1
           Bandage - Flexible Neon  6299165        273         1

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [5]:
cods = pd.pivot_table(data,index='ProductName',columns='CustomerID',values='Quantity',fill_value=0)
cods.head(5)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [6]:
cods.T.head(5)

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [7]:
min_max_norm = lambda serie: (serie - min(serie))/(max(serie)-min(serie))
norm_data = cods.apply(min_max_norm,axis=1)

In [8]:
norm_data.head(10)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.04,0.00,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.04,0.00,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Appetizer - Sausage Rolls,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
Apricots - Dried,0.04,0.0,0.00,0.0,0.04,0.00,0.0,0.00,0.00,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apricots - Halves,0.00,0.0,0.04,0.0,0.00,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
Apricots Fresh,0.00,0.0,0.04,0.0,0.00,0.04,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Arizona - Green Tea,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.04,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Artichokes - Jerusalem,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [9]:
similarities = pd.DataFrame(1/(1 + squareform(pdist(norm_data.T, 'euclidean'))), 
                         index=cods.columns, columns=cods.columns)

similarities.head(5)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.699813,0.721633,0.716343,0.705412,0.711260,0.699813,0.709281,0.707332,0.711260,...,0.107775,0.113757,0.111535,0.119925,0.110006,0.112070,0.109208,0.112133,0.102533,0.109266
200,0.699813,1.000000,0.700730,0.699813,0.685136,0.693571,0.698903,0.695324,0.700730,0.704462,...,0.108099,0.114006,0.111895,0.120457,0.110111,0.112244,0.109782,0.112435,0.102617,0.109193
264,0.721633,0.700730,1.000000,0.707332,0.710267,0.704462,0.700730,0.702583,0.704462,0.704462,...,0.108054,0.113886,0.111531,0.120314,0.110243,0.112705,0.109498,0.112448,0.102868,0.109263
356,0.716343,0.699813,0.707332,1.000000,0.701653,0.703519,0.701653,0.715310,0.701653,0.705412,...,0.107886,0.113821,0.111595,0.120237,0.110004,0.112322,0.109030,0.112449,0.102821,0.109323
412,0.705412,0.685136,0.710267,0.701653,1.000000,0.700730,0.700730,0.710267,0.698903,0.706368,...,0.107765,0.114213,0.111523,0.120305,0.110115,0.112441,0.109139,0.112376,0.102911,0.109668


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [10]:
user_ID = 33
simi_users = similarities.loc[user_ID].sort_values(ascending=False)[1:6]
simi_users

CustomerID
3909    0.724914
264     0.721633
3531    0.721633
2503    0.720557
3305    0.719491
Name: 33, dtype: float64

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [11]:
products_sim_users = q_prod_cus.loc[simi_users.index.to_list()] 
more_freq = products_sim_users.groupby('ProductName').sum().sort_values('Quantity',ascending=False)
more_freq.head(5)

,SalesID,ProductID,Quantity
ProductName,,,
"Salsify, Organic",11534960,1740,4
Quiche Assorted,8659197,696,3
Chocolate - Dark,11544021,540,3
Wine - Charddonnay Errazuriz,13750981,675,3
Bay Leaf,7115943,606,3


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [12]:
# Productos de nuestro usuario target
prod_target = data.loc[data.CustomerID==33].groupby('ProductName').sum().sort_values('Quantity',ascending=False)
possible_recomendations = [product for product in more_freq.index if product not in prod_target.index]
possible_recomendations[:10]

['Chocolate - Dark',
 'Wine - Charddonnay Errazuriz',
 'Bay Leaf',
 'Oranges - Navel, 72',
 'Wine - Ej Gallo Sierra Valley',
 'Pecan Raisin - Tarts',
 'Bread Crumbs - Panko',
 'Cheese - Boursin, Garlic / Herbs',
 'Squid U5 - Thailand',
 'Sauerkraut']

## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [13]:
# Para los productos se genera una matriz de similaridad a partir de 
similarities_prod = pd.DataFrame(1/(1 + squareform(pdist(norm_data, 'euclidean'))), 
                         index=cods.T.columns, columns=cods.T.columns)

similarities_prod.head(3)

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.094638,0.096359,0.101632,0.100118,0.094809,0.095942,0.095817,0.100505,0.106091,...,0.096398,0.097466,0.099970,0.099456,0.097748,0.106352,0.098450,0.096931,0.096140,0.095786
"Appetizer - Mini Egg Roll, Shrimp",0.094638,1.000000,0.095100,0.096587,0.096908,0.091149,0.093284,0.100954,0.099336,0.094652,...,0.094664,0.094571,0.096213,0.096223,0.096515,0.096894,0.096881,0.097837,0.094044,0.095803
Appetizer - Mushroom Tart,0.096359,0.095100,1.000000,0.100020,0.091545,0.094404,0.094788,0.096698,0.098717,0.093634,...,0.095076,0.092008,0.093506,0.102734,0.101633,0.102138,0.097978,0.095418,0.095916,0.095485


### Step 2: Get the products purchased for a specific customer of your choice.

In [14]:
# Productos comprados por el usuario con ID 3909
products_user_3909 = q_prod_cus.loc[3909]
products_user_3909.head(5)

,SalesID,ProductID,Quantity
ProductName,,,
Bay Leaf,6876734,404,2
"Beans - Kidney, Canned",5996386,300,1
Beer - Original Organic Lager,1834282,432,1
Beer - Rickards Red,1525573,284,1
Black Currants,5715812,110,1


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [15]:
def get_similar_products(product):
    return similarities_prod.loc[product].sort_values(ascending=False)[1:6]

list_similar_to_products = [get_similar_products(producto) for producto in products_user_3909.index]
flat_list_prod = [item for sublist in list_similar_to_products for item in sublist.index]
possible_recomendations = [product for product in flat_list_prod if product not in products_user_3909.index]

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [16]:
best_product_recom = pd.DataFrame(possible_recomendations,columns=['Producto'])
best_product_recom['Producto'].value_counts()[:5]

Water - Green Tea Refresher     41
Rabbit - Whole                  34
Scallops 60/80 Iqf              10
Kellogs Special K Cereal         8
Wine - Alsace Gewurztraminer     8
Name: Producto, dtype: int64

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [17]:
full_recommendations = dict()
customerIDs = data.CustomerID.drop_duplicates()
for custID in customerIDs:
    products_user = q_prod_cus.loc[custID]
    list_similar_to_products = [get_similar_products(producto) for producto in products_user.index]
    flat_list_prod = [item for sublist in list_similar_to_products for item in sublist.index]
    possible_recomendations = [product for product in flat_list_prod if product not in products_user.index]
    full_recommendations[str(custID)] = possible_recomendations[:5]

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [19]:
df_recommendation = pd.DataFrame.from_dict(full_recommendations,orient='index',columns=['Product 1','Product 2','Product 3','Product 4','Product 5'])
df_recommendation.head(5)

,Product 1,Product 2,Product 3,Product 4,Product 5
61288,"Doilies - 5, Paper",Beets - Mini Golden,Sauce - Demi Glace,Yeast Dry - Fermipan,Wine - Charddonnay Errazuriz
77352,Anchovy Paste - 56 G Tube,Sauce - Demi Glace,Blueberries,Rabbit - Whole,Chinese Foods - Chicken
40094,Rabbit - Whole,"Nut - Pistachio, Shelled",Cookies Cereal Nut,Whmis - Spray Bottle Trigger,Kellogs Special K Cereal
23548,Rabbit - Whole,Macaroons - Two Bite Choc,Blueberries,"Orange - Canned, Mandarin",Whmis - Spray Bottle Trigger
78981,Water - Green Tea Refresher,Rabbit - Whole,Smirnoff Green Apple Twist,Vol Au Vents,Kellogs Special K Cereal


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [20]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [63]:
new_user_df = pd.DataFrame(new_customer.items(),columns=['Product','-1']).set_index('Product')
new_user_df

,-1
Product,
Cookies - Assorted,3
Flavouring - Orange,3
Fenngreek Seed,1
Wine - White Cab Sauv.on,1
Bandage - Flexible Neon,3
Oil - Shortening - All - Purpose,2
Beef - Montreal Smoked Brisket,4
French Pastry - Mini Chocolate,4
Snapple Lemon Tea,5


In [85]:
# Normalizamos los datos del nuevo usuario.
norm_new_user = new_user_df.apply(min_max_norm)
norm_data['1'] = 0
# Agregamos la información al 
for prod in norm_new_user.index:
    norm_data['1'].loc[prod] = norm_new_user.loc[prod].iloc[-1]

In [89]:
#Calculamos de nuevo las distancias entre usuarios habiendo agregado al nuevo usuario
similarities_update_users = pd.DataFrame(1/(1 + squareform(pdist(norm_data.T, 'euclidean'))), 
                         index=norm_data.columns, columns=norm_data.columns)

user_ID = '1'
simi_users = similarities_update_users.loc[user_ID].sort_values(ascending=False)[1:6]
simi_users

CustomerID
1072    0.322351
1534    0.322212
2566    0.321720
412     0.321563
3472    0.320763
Name: 1, dtype: float64

In [90]:
products_sim_users = q_prod_cus.loc[simi_users.index.to_list()] 
more_freq = products_sim_users.groupby('ProductName').sum().sort_values('Quantity',ascending=False)
more_freq.head(5)

,SalesID,ProductID,Quantity
ProductName,,,
Spinach - Baby,18994044,145,5
Beans - Wax,13864299,132,4
Muffin - Carrot Individual Wrap,18750844,1480,4
Sauce - Demi Glace,10874221,648,4
Soup - Campbells Tomato Ravioli,5929072,1353,3


In [91]:
prod_target = data.loc[data.CustomerID==33].groupby('ProductName').sum().sort_values('Quantity',ascending=False)
possible_recomendations = [product for product in more_freq.index if product not in prod_target.index]
possible_recomendations[:10]

['Spinach - Baby',
 'Beans - Wax',
 'Muffin - Carrot Individual Wrap',
 'Soup - Campbells Tomato Ravioli',
 'Juice - Cranberry, 341 Ml',
 'Cheese - Brie, Triple Creme',
 'Sole - Dover, Whole, Fresh',
 'Curry Paste - Madras',
 'Flavouring - Orange',
 'Mushroom - Porcini, Dry']

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [98]:
# Productos comprados por el usuario nuevo
products_user_new = new_user_df.index
list_similar_to_products = [get_similar_products(producto) for producto in products_user_new]
flat_list_prod = [item for sublist in list_similar_to_products for item in sublist.index]
possible_recomendations = [product for product in flat_list_prod if product not in products_user_new]

In [101]:
best_product_recom = pd.DataFrame(possible_recomendations,columns=['Producto'])
best_product_recom['Producto'].value_counts()[:5]

Water - Green Tea Refresher         5
Wine - Alsace Gewurztraminer        4
Salmon Steak - Cohoe 8 Oz           3
Pork - Loin, Center Cut             3
Beef - Ground, Extra Lean, Fresh    2
Name: Producto, dtype: int64